In [1]:
import pandas as pd

# Read input CSVs
products_df = pd.read_csv("C:\\Users\\SENTHURAN\\Downloads\\products (1).csv")
sales_df = pd.read_csv("C:\\Users\\SENTHURAN\\Downloads\\sales.csv")

In [2]:
display(products_df)
display(sales_df)

,sku,name,current_price,cost_price,stock
0,A123,Item A,649.99,500,150
1,B456,Item B,699.00,550,15
2,C789,Item C,999.00,500,250


,sku,quantity_sold
0,A123,10
1,B456,35
2,C789,0


In [3]:
# Merge dataframes on 'sku'
merged_df = pd.merge(products_df, sales_df, on="sku", how="left")
merged_df["quantity_sold"] = merged_df["quantity_sold"].fillna(0)

In [4]:
display(merged_df)

,sku,name,current_price,cost_price,stock,quantity_sold
0,A123,Item A,649.99,500,150,10
1,B456,Item B,699.00,550,15,35
2,C789,Item C,999.00,500,250,0


In [8]:
def compute_new_price(row):
    
    # Extract values from the row
    current_price = row['current_price']
    cost_price = row['cost_price']
    stock = row['stock']
    quantity_sold = row['quantity_sold']
    
    # Start with the current price as default
    new_price = current_price

    # === Apply pricing rules in order of precedence ===

    # Rule 1: Low Stock, High Demand - Increase price by 15%
    if stock < 20 and quantity_sold > 30:
        new_price = current_price * 1.15

    # Rule 2: Dead Stock - Decrease price by 30%
    elif stock > 200 and quantity_sold == 0:
        new_price = current_price * 0.70

    # Rule 3: Overstocked Inventory - Decrease price by 10%
    elif stock > 100 and quantity_sold < 20:
        new_price = current_price * 0.90

    # === Apply Rule 4: Minimum Profit Constraint (Always applied) ===
    # The new price must be at least 20% above the cost_price.
    minimum_allowed_price = cost_price * 1.2
    if new_price < minimum_allowed_price:
        new_price = minimum_allowed_price

    # Return the final computed price rounded to 2 decimals
    return round(new_price, 2)

# Apply the compute_new_price function across the DataFrame
merged_df['new_price'] = merged_df.apply(compute_new_price, axis=1)

In [9]:
# Convert prices to string with units, e.g., "699.00 INR"
merged_df['old_price'] = merged_df['current_price'].apply(lambda price: f"{round(price, 2)} INR")
merged_df['new_price'] = merged_df['new_price'].apply(lambda price: f"{round(price, 2)} INR")

In [10]:
# Select required columns for the output CSV
output_df = merged_df[['sku', 'old_price', 'new_price']]

In [11]:
# Write to the updated_prices.csv file
output_df.to_csv("C:\\Users\\SENTHURAN\\Downloads\\updated_prices.csv", index=False)

In [12]:
output_df

,sku,old_price,new_price
0,A123,649.99 INR,600.0 INR
1,B456,699.0 INR,803.85 INR
2,C789,999.0 INR,699.3 INR
